##- - 이전 코드 시작 - -

In [1]:
!pip install ratsnlp

     |████████████████████████████████| 42 kB 305 kB/s 
     |████████████████████████████████| 806 kB 9.3 MB/s 
     |████████████████████████████████| 57 kB 3.6 MB/s 
     |████████████████████████████████| 2.8 MB 44.2 MB/s 
     |████████████████████████████████| 408 kB 14.9 MB/s 
     |████████████████████████████████| 829 kB 46.6 MB/s 
     |████████████████████████████████| 636 kB 42.0 MB/s 
     |████████████████████████████████| 136 kB 43.4 MB/s 
     |████████████████████████████████| 895 kB 53.9 MB/s 
     |████████████████████████████████| 77 kB 6.4 MB/s 
     |████████████████████████████████| 3.3 MB 44.1 MB/s 
     |████████████████████████████████| 1.1 MB 44.7 MB/s 
     |████████████████████████████████| 96 kB 5.9 MB/s 
     |████████████████████████████████| 271 kB 37.6 MB/s 
     |████████████████████████████████| 94 kB 676 kB/s 
     |████████████████████████████████| 144 kB 67.1 MB/s 
  Created wheel for future: filename=future-0.18.2-py3-none-any.whl size=491070 sha

In [2]:
#체크포인트 로드
from transformers import GPT2LMHeadModel
model = GPT2LMHeadModel.from_pretrained(
    "skt/kogpt2-base-v2",
)
model.eval()

Downloading:   0%|          | 0.00/1.00k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/513M [00:00<?, ?B/s]

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(51200, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )


In [3]:
#토크나이저 로드
from transformers import PreTrainedTokenizerFast
tokenizer = PreTrainedTokenizerFast.from_pretrained(
    "skt/kogpt2-base-v2",
    eos_token="</s>",
)

Downloading:   0%|          | 0.00/2.83M [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


In [4]:
#모델 입력값 만들기
input_ids = tokenizer.encode("안녕하세요", return_tensors="pt")

In [5]:
#그리드 서치
import torch
with torch.no_grad():
  generated_ids = model.generate(
      input_ids,
      do_sample=False,
      min_length=10,
      max_length=50,
  )

In [6]:
#토큰 인덱스를 문장으로 복원하기
print(tokenizer.decode([el.item() for el in generated_ids[0]]))

안녕하세요?"
"그럼, 그건 뭐예요?"
"그럼, 그건 뭐예요?"
"그럼, 그건 뭐예요?"
"그럼, 그건 뭐예요?"



In [7]:
#빔 서치
with torch.no_grad():
    generated_ids = model.generate(
        input_ids,
        do_sample=False,
        mix_length=10,
        max_length=50,
        num_beams=3,
    )
    print(tokenizer.decode([el.item() for el in generated_ids[0]]))

안녕하세요?"
"그렇지 않습니다."
"그렇지 않습니다."
"그렇지 않습니다."
"그렇지 않습니다."
"그렇지 않습니다."
"그렇지 않습니다."
"그


In [8]:
#반복 줄이기(1)
with torch.no_grad():
    generated_ids = model.generate(
        input_ids,
        do_sample=False,
        min_length=10,
        max_length=50,
        no_repeat_ngram_size=3,
    )
    print(tokenizer.decode([el.item() for el in generated_ids[0]]))

안녕하세요?"
"그럼, 그건 뭐예요?" 하고 나는 물었다.
"그건 뭐죠?" 나는 물었다.
나는 대답하지 않았다.
"그런데 왜 그걸 물어요? 그건 무슨 뜻이에요?


In [9]:
#반복 줄이기(2)
with torch.no_grad():
    generated_ids = model.generate(
        input_ids,
        do_sample=False,
        min_length=10,
        max_length=50,
        repetition_penalty=1.0,
    )
    print(tokenizer.decode([el.item() for el in generated_ids[0]]))

안녕하세요?"
"그럼, 그건 뭐예요?"
"그럼, 그건 뭐예요?"
"그럼, 그건 뭐예요?"
"그럼, 그건 뭐예요?"



In [10]:
#repetition_penalty=1.1
with torch.no_grad():
    generated_ids = model.generate(
        input_ids,
        do_sample=False,
        min_length=10,
        max_length=50,
        repetition_penalty=1.1,
    )
    print(tokenizer.decode([el.item() for el in generated_ids[0]]))

안녕하세요?"
"그럼, 그건 뭐예요?"
"아니요, 저는요."
"그럼, 그건 무슨 말씀이신지요?"
"그럼, 그건 뭐예요?"



In [11]:
#repetition_penalty=1.2
with torch.no_grad():
    generated_ids = model.generate(
        input_ids,
        do_sample=False,
        min_length=10,
        max_length=50,
        repetition_penalty=1.2,
    )
    print(tokenizer.decode([el.item() for el in generated_ids[0]]))

안녕하세요?"
"그럼, 그건 뭐예요, 아저씨. 저는 지금 이 순간에도 괜찮아요."
"그래서 오늘은 제가 할 수 있는 일이 무엇인지 말해 보겠습니다."
"이제


In [12]:
#repetition_penalty=1.5
with torch.no_grad():
    generated_ids = model.generate(
        input_ids,
        do_sample=False,
        min_length=10,
        max_length=50,
        repetition_penalty=1.5,
    )
    print(tokenizer.decode([el.item() for el in generated_ids[0]]))

안녕하세요?"
"그럼, 그건 뭐예요, 아저씨. 저는 지금 이 순간에도 괜찮아요. 그리고 제가 할 수 있는 일은 아무것도 없어요.
이제 그만 돌아가고 싶어요.
제가 하는 일이 무엇


In [13]:
#탑-k 샘플링 1try
with torch.no_grad():
  generated_ids = model.generate(
      input_ids,
      do_sample=True,
      min_length=10,
      max_length=50,
      top_k=50,
  )
  print(tokenizer.decode([el.item() for el in generated_ids[0]]))

안녕하세요’ 이벤트를 오는 2월19일까지 진행한다.
이벤트 기간 중에는 ‘화이트댄스쇼’ ‘트램퍼싱’ 등 다양한 이벤트가 진행된다.</d> 인천시에 따르면 시는 지난달 말 인천시청에서 송


In [14]:
#탑-k 샘플링 2try
with torch.no_grad():
  generated_ids = model.generate(
      input_ids,
      do_sample=True,
      min_length=10,
      max_length=50,
      top_k=50,
  )
  print(tokenizer.decode([el.item() for el in generated_ids[0]]))

안녕하세요’를 부르짖고 나섰다.
‘한겨레’ 지면을 장식한 것은 그 자체로 뜨거운 ‘사랑의 열기’가 아닐 수 없다.
지난 14일 최창식 목사는 ‘최진실 뉴스데스크’를 통해 “서울 시민들은


##- - 이전 코드 끝 - -

### 템퍼러처 스케일링


***템퍼러처 스케일링(temperature scaling)*** 이란 모델의 다음 토큰 확률 분포를 대소 관계의 역전 없이 분포의 모양만을 바꿔서 문장을 다양하게 생성하는 기법입니다.   
다음 그림이 템퍼러처 스케일링을 적용한 예시입니다.

<center><그림 - 템퍼러처 스케일링></center>

<p align="center"><img src="https://i.imgur.com/2Lb8CGj.jpg">


원래대로라면 `그` 다음 토큰 확률은 각각 `책`(0.5), `집`(0.4), `사람`(0.1)이었습니다.   
템퍼러처 스케일링을 적용한 결과 그 확률이 `책`(0.75), `집`(0.23), `사람`(0.02)으로 바뀌었습니다. 마찬가지로 `그 책` 다음 토큰 확률도 각각 `이`(0.4), `을`(0.3), `읽`(0.3)에서 `이`(0.6), `을`(0.2), `읽`(0.2)으로 바뀌었습니다. 순위는 변하지 않았지만 원래 컸던 확률은 더 커지고, 작았던 확률은 더 작아져 확률 분포의 모양이 뾰족(sharp)해졌음을 알 수 있습니다.

템퍼러처 스케일링은 모델의 출력 로짓(소프트맥스 변환 전 벡터)의 모든 요솟값을 `temperature`로 나누는 방식으로 적용합니다. 예를 들어 로짓이 [-1.0 2.0 3.0]이고 `temperature` 가 2라면 템퍼러처 스케일링 적용 후 로짓은 [-0.5 1.0 1.5]가 됩니다. 여기에 소프트맥스를 취해 다음 단어 확률 분포로 만듭니다. `temperature`가 1이면 전혀 변화가 생기지 않습니다. 음수라면 단어들 사이의 대소 관계가 완전히 뒤바뀌므로 0보다 큰 값을 지정해야 합니다.

다음 코드는 탑-k 샘플링에 템퍼러처 스케일링을 적용한 코드입니다.  
다음 단어를 선택할 때 확률값이 높은 50개에서 고르되, 그 확률값은 템퍼러처 스케일링(`temperature=0.01`)으로 바꾼다는 뜻입니다.

In [17]:
#템퍼러처 스케일링
with torch.no_grad():
    generated_ids = model.generate(
        input_ids,
        do_sample=True,
        min_length=10,
        max_length=50,
        temperature=0.01,
    )
    print(tokenizer.decode([el.item() for el in generated_ids[0]]))

안녕하세요?"
"그럼, 그건 뭐예요?"
"그럼, 그건 뭐예요?"
"그럼, 그건 뭐예요?"
"그럼, 그건 뭐예요?"



코드를 실행해 보니 탑-k 샘플링을 수행했음에도 그리디 서치로 생성한 문장과 같습니다.

이 코드에서는 `temperature`를 0.01로 설정했는데, 이 값이 0에 가까울수록 확률 분포 모양이 원래보다 뾰족해집니다. 확률 분포 모양이 뾰족하다는 말은 원래 컸던 확률은 더 커지고 작았던 확률은 더 작아진다는 의미입니다. 그만큼 확률값 기준 1등 토큰이 다음 토큰으로 뽑힐 가능성이 커진다는 이야기입니다.   

`temperature`를 1보다 큰 값으로 설정한다면 확률 분포가 평평해집니다(uniform). 원래 컸던 확률과 작았던 확률 사이의 차이가 줄어든다는 이야기입니다.  
바꿔 말하자면 확률값이 작아서(=모델이 다음 토큰으로 부적절하다고 예측) 기존 탑-k 샘플링에선 선택되기 어려웠던 토큰들이 다음 토큰으로 선택될 수 있습니다. 그만큼 다양한 문장이 생성될 가능성이 높아지지만 생성 문장의 품질이 나빠질 수 있습니다.  

다음은 앞 코드에서 `temperature`값을 100000000.0으로 설정한 결과입니다.

In [18]:
#템퍼러처 스케일링(100000000.0)
with torch.no_grad():
    generated_ids = model.generate(
        input_ids,
        do_sample=True,
        min_length=10,
        max_length=50,
        temperature=100000000.0,
    )
    print(tokenizer.decode([el.item() for el in generated_ids[0]]))

안녕하세요라는 편지를 전해왔다.
서운하기도 했다는 서씨가 이런 이야기를 하니 갑자기 화가 일어났다.
그로부터 약 몇 달 전의 일이었다.
여중 3반은 학생에게, 혹은 방생교육을 마치고서였대야만 한다고 그는 털어 버렸다.
자네가 나를


요약하자면 `temperature`를 1보다 작게하면 상대적으로 정확한 문장을, 1보다 크게 하면 상대적으로 다양한 문장을 생성할 수 있습니다. 템퍼러처 스케일링은 탑-k 샘플링, 탑-p 샘플링과 함께 적용해야 의미가 있습니다. 이어서 탑-p 샘플링 설명이 나옵니다.

### 탑-p 샘플링

***탑-p 샘플링(top-p sampling)*** 은 확률값이 높은 순서대로 내림차순 정렬을 한 뒤 누적 확률값이 $p$ 이상인 최소 개수의 토큰 집합 가운데 하나를 다음 토큰으로 선택하는 기법입니다. ***뉴클리어스 샘플링(necleus sampling)*** 이라고도 부릅니다. 확률값을 기준으로 토큰을 내림차순 정렬해 그 값이 높은 토큰들을 다음 토큰 후보로 삼는다는 점에서는 탑-k 샘플링과 같지만, 상위 $k$개를 후보로 삼느냐(탑-k 샘플링), 누적 확률값이 $p$ 이상인 최소 개수의 토큰들을 후보로 삼느냐(탑-p 샘플링)에 따라 차이가 있습니다.

다음 그림은 `그`라는 컨텍스트를 입력했을 때 모델이 출력한 다음 토큰의 확률 분포입니다. $p$를 0.92로 설정했다고 가정해 봅시다. 가장 높은 확률을 가지는 토큰부터 시작해 누적 확률값이 $p$ 이상이 될 때까지 하나씩 순서대로 다음 토큰 후보에 추가합니다. 다음 그림에서 다음 단어 후보는 `책`부터 `회사`까지 9개, 그리고 이들의 누적 확률합은 0.94가 되는 것을 확인할 수 있습니다. $k$가 6인 탑-k 샘플링에서는 다음 단어 후보가 `책`부터 `의자`까지 6개, 그리고 이들의 누적 확률합은 0.68였던 것과 비교할 수 있습니다.

<center><그림 - 탑-p 샘플링(1)></center>

<p align="center"><img src="https://i.imgur.com/ZEQoJU2.jpg">

다음 그림은 `그 사람`이라는 컨텍스트를 입력했을 때 모델이 출력한 다음 토큰 확률 분포입니다. $p$를 0.92로 설정했을 때 다음 토큰 후보는 `처럼`부터 `누구`까지 3개, 그리고 이들의 누적 확률합은 0.97인 것을 확인할 수 있습니다. $k$가 6인 탑-k 샘플링에서는 다음 토큰 후보가 `처럼`부터 `과`까지 6개, 그리고 이들의 누적 확률합은 0.99였던 것과 비교할 수 있습니다.

<center><그림 - 탑-p 샘플링(2)></center>

<p align="center"><img src="https://i.imgur.com/WjpElDc.jpg">

탑-p 샘플링은 누적 확률합으로 후보 토큰을 취하기 때문에 누적 확률합이 매 순간 비슷한 반면 후보 토큰 개수는 해당 분포에 따라 달라집니다. 반대로 탑-k 샘플링은 토큰 개수로 후보 토큰을 취하기 때문에 후보 토큰 개수는 일정한 반면 누적 확률합은 해당 분포에 따라 달라집니다.  
다만 둘 모두 확률값이 낮은 토큰은 다음 토큰 후보에서 빠지므로 품질 높은 문장을 생성할 가능성을 높입니다.

다음 코드는 탑-p 샘플링을 수행합니다. 핵심 인자는 `do_sample=True`, `top_p=0.92`입니다.  
샘플링 방식으로 다음 단어를 선택하되 $p$를 0.92로 설정한다는 뜻입니다. `top_p`에는 0~1 사이의 실수를 입력해야 합니다.

In [19]:
#탑-p 샘플링(1)
with torch.no_grad():
    generated_ids = model.generate(
        input_ids,
        do_sample=True,
        min_length=10,
        max_length=50,
        top_p=0.92,
    )
    print(tokenizer.decode([el.item() for el in generated_ids[0]]))

안녕하세요?' 라는 말이 나오면 "아니 그냥 그냥 '~했습니다'라는 것이죠"라고 대답합니다.
하지만 그걸 두고 "내가 저런 말을 못해서"라는 이야기가 들리면 "이거 내말이다" 라고 대답합니다.
물론 '


한편 `top_p`를 1.0으로 설정한다면 확률값이 낮은 단어를 전혀 배제하지 않고, 다음 단어 후보로 전체 어휘를 고려한다는 의미가 됩니다. `top_p`가 0에 가까울수록 후보 토큰이 줄어들어 확률값이 높은 토큰들만 남게 돼 그리디 서치와 비슷해집니다. 


다음은 앞 코드에서 `top_p`를 0.01로 설정한 결과입니다.   
탑-p 샘플링을 수행했음에도 그리디 서치 결과와 같음을 확인할 수 있습니다.

In [20]:
#탑-p 샘플링(1) - 0.01
with torch.no_grad():
    generated_ids = model.generate(
        input_ids,
        do_sample=True,
        min_length=10,
        max_length=50,
        top_p=0.01,
    )
    print(tokenizer.decode([el.item() for el in generated_ids[0]]))

안녕하세요?"
"그럼, 그건 뭐예요?"
"그럼, 그건 뭐예요?"
"그럼, 그건 뭐예요?"
"그럼, 그건 뭐예요?"



### 종합 적용하기

지금까지 설명한 문장 생성 방식을 모두 종합해 적용해 봅시다.  
그리디 서치나 빔 서치는 가장 높은 확률값을 내는 문장을 생성해주지만, 컨텍스트가 같을 때는 매번 같은 문장이 나오므로 샘플링 방식을 적용하겠습니다. 그리고 생성된 문장이 너무 짧거나 길지 않게 하고 반복되는 토큰을 될 수 있으면 배제하며 템퍼러처 스케일링으로 원래 확률 분포를 조금 뾰족하게 해 확률값이 높은 토큰이 더 잘 나오도록 하겠습니다.   


마지막으로 탑-k 샘플링과 탑-p 샘플링을 동시에 적용해 확률값이 낮은 토큰들은 후보 단어에서 제외하겠습니다.  
다음은 이러한 내용을 적용한 코드입니다.

In [21]:
#종합 적용
with torch.no_grad():
  generated_ids = model.generate(
      input_ids,
      do_sample=True,
      min_length=10,
      max_length=50,
      repetition_penalty=1.5,
      no_repeat_ngram_size=3,
      temperature=0.9,
      top_k=50,
      top_p=0.92,
  )
  print(tokenizer.decode([el.item() for el in generated_ids[0]]))

안녕하세요?"하고 물었다.
그러자 유준은 "아니요, 저는 그렇게 생각하지 않는다"고 대답했다.
이어 그는 잠시 말을 멈추고 웃더니 다시 입을 열었다.
유준이 한 대 맞췄는데, 나는 그때도 난생처음 들은


역시 코드를 실행할 때마다 다른 문장이 생성됩니다.

### 맺음말
---
이번 절에서는 프리트레인을 마친 언어 모델을 가지고 문장을 생성하는 방법을 실습해 보았습니다.   
다음 단어 후보들을 모두 탐색하는 것은 불가능에 가까우므로 그리디 서치, 빔 서치, 탑-k 샘플링, 탑-p 샘플링 등 기법이 제안되었습니다.